In [43]:
import requests
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import klib
from pint import UnitRegistry

ureg = UnitRegistry()
ureg.default_format = '.2f'
ureg.load_definitions('crypto_mining_units.txt')

pd.set_option("display.max.columns", None)


In [46]:
def get_network_data(asset: str, metrics: list = ['HashRate', 'IssTotUSD', 'FeeTotUSD', 'PriceUSD', 'FeeMeanNtv'], end_time: str = date.today().strftime('%Y-%m-%d')):
    api_str = f"https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?assets={asset}&metrics={','.join(metrics)}&end_time={end_time}&page_size=10000&pretty=true"
    print(api_str)
    response = requests.get(api_str).json()
    return pd.DataFrame(response['data'])

In [47]:
df_unclean = get_network_data("btc", ['HashRate', 'PriceUSD', 'FeeMeanNtv']).fillna("0")

df_cleaned = klib.data_cleaning(df_unclean)

df_cleaned.time = df_cleaned.time.str.replace("T00:00:00.000000000Z", "")
df_cleaned.time = pd.to_datetime(df_cleaned.time)

df_cleaned[['hash_rate', 'price_usd']] =  df_cleaned[['hash_rate', 'price_usd']].apply(pd.to_numeric)

df_cleaned.index = df_cleaned.time
df_cleaned.drop(columns=['time'], inplace=True)

df_cleaned.dtypes
df = df_cleaned
df.to_csv("hell")
df

https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?assets=btc&metrics=HashRate,PriceUSD,FeeMeanNtv&end_time=2021-07-12&page_size=10000&pretty=true


KeyError: 'data'

In [39]:
def get_latest_price():
    return float(df.price_usd.tail(1).values)
def get_latest_hashrate():
    return float(df.hash_rate.tail(1).values)
def get_latest_tx_fees():
    return float(df.fee_mean_ntv.tail(1).values[0])

https://www.aniccaresearch.tech/blog/the-alchemy-of-hashpower-part-i

D = C / [P * S/H * (m+n) * 6 * 24 - k * S * r * 24]

In [44]:
# Calculating days to break even for one S19j Pro 
capex       = 5_500 * ureg.USD
opex        = 0.1   * ureg.USD / ureg.kWh
asset_price = get_latest_price() * ureg.USD / ureg.BTC

miner_hashrate   = 96 * ureg.TH / ureg.second  
network_hashrate = get_latest_hashrate() * ureg.TH / ureg.s
miner_efficency  = 29.5 * ureg.J / ureg.TH 
hashrate_share   = miner_hashrate.to('hashes / second') / network_hashrate.to('hashes / second')

subsidy      = 6.25 * ureg.BTC / ureg.BTC_block
tx_fees      = get_latest_tx_fees() * ureg.BTC / ureg.BTC_block
block_reward = subsidy + tx_fees
blocks       = 144 * ureg.BTC_block / ureg.day

profits    = asset_price * hashrate_share * block_reward * blocks
losses     = miner_efficency.to('kWh / terahashes') * miner_hashrate.to('terahashes / day') * opex 
break_even = capex / (profits - losses)

print(f'{break_even} to break even with one S19j Pro with Income {profits} and Electricity Costs {losses}')


208.58 day to break even with one S19j Pro with Income 33.17 USD / day and Electricity Costs 6.80 USD / day
